In [1]:
cd ../../../../

/Users/mykhailoslukvin/repo/dv-data-pipeline


/Users/mykhailoslukvin/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import asyncio
import time
import pandas as pd
from pprint import pprint as print
from tqdm.asyncio import tqdm_asyncio

from dfpp.transformation.source_notebooks.ilo_org.geo_utils import get_iso3_map
from dfpp.transformation.source_notebooks.ilo_org.publish import publish_indicator
from dfpp.transformation.source_notebooks.ilo_org.retrieve import (
    get_codebook,
    get_indicator,
    list_indicators,
)
from dfpp.transformation.source_notebooks.ilo_org.transform import (
    sanitize_categories,
    transform_indicator,
)

MAX_CONCURRENCY = 1

In [3]:
ISO_3_MAP = await get_iso3_map()

In [4]:
df_classif1, df_classif2 = get_codebook()

In [9]:
df_classif1, df_classif2 = sanitize_categories(df_classif1, df_classif2)

In [12]:
indicator_list = list_indicators()
df_indicators = pd.DataFrame(indicator_list)
df_annual_indicators = df_indicators[df_indicators.freq == "A"]
assert (
    df_annual_indicators.id.value_counts().max() == 1
), "Each indicator must have one record"

In [13]:
async def process_indicator(semaphore, indicator, df_classif1, df_classif2, ISO_3_MAP):
    try:
        indicator_id = indicator["id"]

        async with semaphore:
            data = await get_indicator(indicator["id"])
            time.sleep(5)
            df_source = pd.DataFrame(data)
            assert df_source.shape[0] > 0, f"The {indicator_id} DataFrame is empty"

            assert (
                abs(df_source.shape[0] - indicator["n_records"]) == 0
            ), f"Shape mismatch with expected dimensions expected {indicator['n_records']} but got {df_source.shape[0]}"
            print(indicator["indicator_label"])

            df = df_source.copy()

            df_indicator = transform_indicator(df, df_classif1, df_classif2, ISO_3_MAP)

            await publish_indicator(indicator_id, df_indicator)

    except Exception as e:
        return indicator_id, e


async def process_all_indicators(
    df_annual_indicators, df_classif1, df_classif2, ISO_3_MAP
):

    semaphore = asyncio.Semaphore(MAX_CONCURRENCY)
    failed_indicators = []

    indicators = df_annual_indicators.to_dict(orient="records")

    tasks = [
        process_indicator(semaphore, indicator, df_classif1, df_classif2, ISO_3_MAP)
        for indicator in indicators
    ]

    for future in tqdm_asyncio.as_completed(tasks):
        result = await future
        if isinstance(result, tuple) and len(result) == 2:
            failed_indicators.append(result)

    return failed_indicators

In [14]:
failed_indicators = await process_all_indicators(
    df_annual_indicators, df_classif1, df_classif2, ISO_3_MAP
)

  0%|          | 0/5 [00:00<?, ?it/s]

'Share of children in hazardous work by sex, age and school attendance (%)'


 20%|██        | 1/5 [00:06<00:25,  6.42s/it]

'Share of children in child labour by sex, age and and rural / urban areas (%)'


 40%|████      | 2/5 [00:13<00:19,  6.53s/it]

...

 60%|██████    | 3/5 [00:19<00:12,  6.32s/it]

'Share of children in hazardous work by sex, age and rural / urban areas (%)'


 80%|████████  | 4/5 [00:25<00:06,  6.39s/it]

'Share of children in child labour by sex, age and school attendance (%)'


100%|██████████| 5/5 [00:32<00:00,  6.46s/it]


In [15]:
assert len(failed_indicators) == 0, print(failed_indicators)